# image_detector

Image detection module using YOLOv8 for detecting humans and vehicles in satellite imagery.

In [ ]:
import cv2
import numpy as np
import pandas as pd
import torch
from ultralytics import YOLO
import os
from utils import save_dataframe, generate_image_id

## Code

In [ ]:
# Image detection module using YOLOv8 for detecting humans and vehicles in satellite imagery.

class ImageDetector:
    """Image detection using YOLO for satellite imagery."""
    
    def __init__(self, model_path='yolov8n.pt'):
        """
        Initialize the image detector.
        Args:
            model_path: Path to YOLO model weights
        """
        print("Initializing YOLO model...")
        self.model = YOLO(model_path)
        # COCO class names for person and vehicle detection
        self.class_names = {
            0: 'person',
            1: 'bicycle', 
            2: 'car',
            3: 'motorcycle',
            5: 'bus',
            7: 'truck'
        }
        self.target_classes = list(self.class_names.keys())
        
    def detect_objects(self, image_path, confidence_threshold=0.3):
        """
        Detect humans and vehicles in an image.
        
        Args:
            image_path: Path to the image file
            confidence_threshold: Minimum confidence for detections
            
        Returns:
            List of detection dictionaries
        """
        # Read image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Warning: Could not read image {image_path}")
            return []
        
        # Run detection
        results = self.model(image, verbose=False)
        
        detections = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                # Get class and confidence
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                
                # Filter for target classes and confidence
                if cls in self.target_classes and conf >= confidence_threshold:
                    # Get bounding box coordinates
                    xyxy = box.xyxy[0].tolist()
                    
                    detections.append({
                        'class_id': cls,
                        'class_name': self.class_names.get(cls, 'unknown'),
                        'confidence': conf,
                        'bbox': xyxy
                    })
        
        return detections
    
    def process_image_batch(self, image_metadata_df, output_dir='output'):
        """
        Process a batch of images and save results.
        
        Args:
            image_metadata_df: DataFrame with image metadata
            output_dir: Directory to save results
            
        Returns:
            DataFrame with detection results
        """
        all_detections = []
        
        print(f"Processing {len(image_metadata_df)} images...")
        for idx, row in image_metadata_df.iterrows():
            image_path = row['image_path']
            image_id = row['image_id']
            
            # Detect objects
            detections = self.detect_objects(image_path)
            
            # Add metadata to each detection
            for det in detections:
                all_detections.append({
                    'image_id': image_id,
                    'image_path': image_path,
                    'latitude': row['latitude'],
                    'longitude': row['longitude'],
                    'timestamp': row['timestamp'],
                    'class_id': det['class_id'],
                    'class_name': det['class_name'],
                    'confidence': det['confidence'],
                    'bbox_x1': det['bbox'][0],
                    'bbox_y1': det['bbox'][1],
                    'bbox_x2': det['bbox'][2],
                    'bbox_y2': det['bbox'][3]
                })
        
        # Convert to DataFrame
        detections_df = pd.DataFrame(all_detections)
        
        # Save results
        save_dataframe(detections_df, 'image_detections.csv', output_dir)
        
        return detections_df
    
    def get_detection_summary(self, detections_df):
        """
        Get summary statistics for detections.
        
        Args:
            detections_df: DataFrame with detection results
            
        Returns:
            Dictionary with summary statistics
        """
        summary = {
            'total_detections': len(detections_df),
            'human_detections': len(detections_df[detections_df['class_name'] == 'person']),
            'vehicle_detections': len(detections_df[detections_df['class_name'] != 'person']),
            'unique_images': detections_df['image_id'].nunique() if len(detections_df) > 0 else 0
        }
        
        return summary


## Test Code

In [ ]:
    # Test the image detector
    detector = ImageDetector()
    # Load sample image metadata
    if os.path.exists('output/image_metadata.csv'):
        metadata_df = pd.read_csv('output/image_metadata.csv')
        detections = detector.process_image_batch(metadata_df)
        summary = detector.get_detection_summary(detections)
        print("Detection Summary:", summary)
    else:
        print("No image metadata found. Run data_generator.py first.")